# SeqAcademy Multiomics Tutorial

# Contents
1. Installation
    1. Set up channels
    2. Create an environment and install the packages
2. Alignment
    1. HISAT 
    2. Samtools
    3. MultiQC
3. ChIP-Seq
    1. MACS
    2. Bedtools
    3. IGV
4. RNA-Seq
    1. HTSeq
    2. DESeq
    3. Visualization

# Installation

Before running any programs, we'll make sure that each software is installed correctly. This tutorial uses Bioconda (https://bioconda.github.io/). Bioconda is a channel for the conda package manager specializing in bioinformatics software. The available packages are listed here: https://bioconda.github.io/recipes.html#recipes.

## Set up channels

You will need to add the bioconda channel as well as the other channels bioconda depends on. It is important to add them in this order so that the priority is set correctly (that is, bioconda is highest priority).

The conda-forge channel contains many general-purpose packages not already found in the defaults channel. The r channel is only included due to backward compatibility. It is not mandatory, but without the r channel packages compiled against R 3.3.1 might not work.

This tutorial uses cells written in python and unix to perform its analyses. Lines that are written in unix are prefixed by an exclamation point. 

Select the following cell and run it. To run a cell, select the cell, click "Cell" the upper taskbar, and select "Run Cells". Or click the cell and press shift + enter.

Alternatively, the contents of any cell may be copy+pasted into the terminal emulator to run.

In [ ]:
!conda config --add channels defaults
!conda config --add channels conda-forge
!conda config --add channels bioconda

## Create an environment and install the packages

In this tutorial we will create an environment named "tutorial" and install the packages in there. Environments offer ways of installing packages in specific environments so they can be managed and run for different specifications. You can create, export, list, remove and update environments that have different versions of Python and/or packages installed in them. Switching or moving between environments is called activating the environment. You can also share an environment file.

This command will create an environment "tutorial" in which to install the packages used in this tutorial.

Run the following commands to create the environment. When it asks if you want to proceed, type "y" and press enter.

This will most likely take 10-15 minutes.

In [ ]:
!conda create -n tutorial hisat2 multiqc macs2 bioconductor-deseq matplotlib ggplot samtools bioconductor-rsamtools bedtools htseq  --yes

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /data/Lei_student/Hussain/miniconda/envs/tutorial:

The following NEW packages will be INSTALLED:

    asn1crypto:                    0.24.0-py27_0               conda-forge
    backports:                     1.0-py27_1                  conda-forge
    backports.functools_lru_cache: 1.5-py27_0                  conda-forge
    backports_abc:                 0.5-py27_0                  conda-forge
    bcftools:                      1.8-1                       bioconda   
    bedtools:                      2.27.1-1                    bioconda   
    bioconductor-annotate:         1.56.0-r3.4.1_0             bioconda   
    bioconductor-annotationdbi:    1.40.0-r3.4.1_0             bioconda   
    bioconductor-biobase:          2.38.0-r3.4.1_0             bioconda   
    bioconductor-biocgenerics:     0.24.0-r3.4.1_0             bioconda   
    bioconductor-biocparallel:   

ca-certificate 100% |################################| Time: 0:00:00   2.02 MB/s
krb5-1.14.6-0. 100% |################################| Time: 0:00:00   6.35 MB/s#                             | Time: 0:00:00   5.71 MB/s MB/s       | Time: 0:00:00   5.43 MB/s  5.56 MB/s             | Time: 0:00:00   5.67 MB/s00:00   5.76 MB/s##                 | Time: 0:00:00   5.84 MB/sme: 0:00:00   5.97 MB/s########################       | Time: 0:00:00   6.21 MB/s3 MB/s93% |#############################   | Time: 0:00:00   6.32 MB/s
libdeflate-0.8 100% |################################| Time: 0:00:00   7.89 MB/s
openssl-1.0.2o 100% |################################| Time: 0:00:00  10.14 MB/s | Time: 0:00:00   9.12 MB/s MB/s  9.10 MB/s.2o  44% |##############                  | Time: 0:00:00   9.25 MB/s####               | Time: 0:00:00   9.20 MB/s################         | Time: 0:00:00   9.68 MB/s###########################    | Time: 0:00:00  10.03 MB/s
curl-7.59.0-1. 100% |#########################

Then activate the environment with the following command.

In [ ]:
# For Mac and Linux

!source activate tutorial

In [ ]:
# For Windows

!activate tutorial

# Alignment
## HISAT (Hierarchical Indexing for Spliced Alignment of Transcripts)

In this tutorial, we'll use Hisat to align the sample reads to a reference genome. Hisat automatically downloads and preprocesses the reads so they're ready to be aligned. Hisat (hierarchical indexing for spliced alignment of transcripts) is a highly efficient system for aligning reads from RNA sequencing experiments. HISAT uses an indexing scheme based on the Burrows-Wheeler transform and the Ferragina-Manzini (FM) index, employing two types of indexes for alignment: a whole-genome FM index to anchor each alignment and numerous local FM indexes for very rapid extensions of these alignments. HISAT’s hierarchical index for the human genome contains 48,000 local FM indexes, each representing a genomic region of ~64,000 bp.

The RNA-Seq data we'll use is from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP106028 and the ChIP-Seq data is from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP132584

The model organism for this project is Yeast i.e. Saccharomyces cerevisiae. For RNA-Seq, yeast data between euploid and aneuoploid conditions will be compared. For ChIP-SEq, yeast data between 3AT-treated and untreated conditions will be compared.

The following cell contains python code. 

The Python programming language comes with a variety of built-in functions. Among these are several common functions, including:

+ print() which prints expressions out
+ abs() which returns the absolute value of a number
+ int() which converts another data type to an integer
+ len() which returns the length of a sequence or collection

These built-in functions, however, are limited, and we can make use of modules to make more sophisticated programs.

Modules are Python .py files that consist of Python code. Any Python file can be referenced as a module. A Python file called hello.py has the module name of hello that can be imported into other Python files or used on the Python command line interpreter. 

Run the following cell to see what it does and observe the output

In [ ]:
from pandas import read_csv

RNASeqSRARunTableFile='data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"]).astype(list)
RNASeqoutputSam = "test/" + RNASeqoutrun + ".sam"
RNASeqoutputAlignmentSummary = "test/" + RNASeqoutrun + ".txt"
RNASeqoutputMetrics = "test/" + RNASeqoutrun + ".metrics"
RNASeqoutputSortBam = "test/" + RNASeqoutrun + ".sorted.bam"

ChIPSeqSRARunTableFile='data/ChIPSeqSRA.tsv'
ChIPSeqSRATable = read_csv(ChIPSeqSRARunTableFile, delimiter='\t')
ChIPSeqoutrun = (ChIPSeqSRATable["Run"]).astype(list)
ChIPSeqoutputSam = "test/" + ChIPSeqoutrun + ".sam"
ChIPSeqoutputAlignmentSummary = "test/" + ChIPSeqoutrun + ".txt"
ChIPSeqoutputMetrics = "test/" + ChIPSeqoutrun + ".metrics"
ChIPSeqoutputSortBam = "test/" + ChIPSeqoutrun + ".sorted.bam"

print("RNA-Seq run " + RNASeqoutrun)
print("ChIP-Seq run " + ChIPSeqoutrun)

Then run the following command to create the yeast index. The following command is a bash script `makeYeastIndex.sh` in the directory `scripts`. When run, the script downloads sequences for the latest Yeast release from Ensembl. By default, it builds and index for just the base files, since alignments to those sequences are the most useful.  To change which categories are built by this script, edit the CHRS_TO_INDEX variable in the `scripts/makeYeastIndex.sh` file. 

In [ ]:
!bash scripts/makeYeastIndex.sh

Align the RNA-Seq samples using Hisat.

This step will most likely take several hours. To ensure that it runs successfully, we are running hisat using `nohup`  to allow it to run even after you logout. The output from this step will be saved in a file `RUN.hisat2.log` in which RUN is the SRR ID of the run. 

In [ ]:
for index, individual in enumerate(RNASeqoutrun):
    run = RNASeqoutrun[index]
    summary = RNASeqoutputAlignmentSummary[index] 
    metrics = RNASeqoutputMetrics[index]
    sam = RNASeqoutputSam[index]
    bam = RNASeqoutputSortBam[index]
    !nohup hisat2 -x yeast_index/genome --sra-acc $run --new-summary --summary-file $summary --met-file $metrics -S $sam & > $run.hisat2.log

## Samtools 

We'll use samtools to sort the output files and convert them to bam files.

Sort the output files and convert them to bam files. Again, nohup is used to ensure it continues running.

In [ ]:
for index, individual in enumerate(RNASeqoutrun):
    run = RNASeqoutrun[index]
    summary = RNASeqoutputAlignmentSummary[index] 
    metrics = RNASeqoutputMetrics[index]
    sam = RNASeqoutputSam[index]
    bam = RNASeqoutputSortBam[index]
    !nohup samtools view -bSF4 $sam | samtools sort -o $bam & > $bam.out
    

Do the same thing for ChIP-Seq samples.

In [ ]:
for index, individual in enumerate(ChIPSeqoutrun):
    run = ChIPSeqoutrun[index]
    summary = ChIPSeqoutputAlignmentSummary[index] 
    metrics = ChIPSeqoutputMetrics[index]
    sam = ChIPSeqoutputSam[index]
    bam = ChIPSeqoutputSortBam[index]
    index = "yeast_index/genome"
    !nohup hisat2 -x $index --sra-acc $run --new-summary --summary-file $summary --met-file $metrics -S $sam & > $run.hisat2.log

In [ ]:
for index, individual in enumerate(ChIPSeqoutrun):
    run = ChIPSeqoutrun[index]
    summary = ChIPSeqoutputAlignmentSummary[index] 
    metrics = ChIPSeqoutputMetrics[index]
    sam = ChIPSeqoutputSam[index]
    bam = ChIPSeqoutputSortBam[index]
    !samtools view -bSF4 $sam | samtools sort -o $bam

## MultiQC

This section details quality control checks on the read data from either RNAseq or ChIPseq data using MultiQC. MultiQC takes all output and log files from an alignment software program and aggregates the information from all samples into one convenient report (html by default).

MultiQC was installed earlier in the tutorial, so all we need to do is run it on the data.

MultiQC is configured to run the same no matter what type of sequencing data is available, therefore the same command can be used to analyze either our RNAseq data or our ChIPseq data.  We include the option 'hisat_output' since we are aligning using the HISAT2 program.  See http://multiqc.info/docs/ for more information.

We use the 'hisat_output' option because we are analyzing data downloaded and aligned using the HISAT2 program.  We use the '--force' option to overwrite any previous versions of the multiqc_report.  '--quiet' only shows log warnings.

In [ ]:
!multiqc "".join(RNASeqoutrun) --quiet --outdir test/multiqc_rnaseq --force
!multiqc "".join(ChIPSeqoutrun) --quiet --outdir test/multiqc_chipseq --force

# ChIP-Seq
## MACS (Model-based Analysis for ChIP-Seq)

Peak-calling is one of the main steps scientists use in determining the locations where protein is bound in DNA. Peak detection software, such as MACS (Model-Based Analysis for ChIP-Seq), call peaks using the aligned sequecnes as input and returns precise locations of predicted peaks as output. In this tutorial, we'll use MACS.

More information about MACS: http://liulab.dfci.harvard.edu/MACS/Download.html

In [ ]:
ChIPSeqControl = ChIPSeqSRATable.loc[ChIPSeqSRATable["source_name"] == "Untreated"]["Run"].astype(list)
ChIPSeqTreatment = ChIPSeqSRATable.loc[ChIPSeqSRATable["source_name"] != "Untreated"]["Run"].astype(list)

print("Control sample " + ChIPSeqControl)
print("Treatment sample " + ChIPSeqTreatment)

In [ ]:
for index, individual in enumerate(ChIPSeqControl):
    outputdirectory = "test/" + ChIPSeqTreatment.iloc[index]
    name = ChIPSeqTreatment.iloc[index]
    immunoprecipitate = "test/" + ChIPSeqTreatment.iloc[index] + ".sorted.bam"
    control = "test/" + ChIPSeqControl.iloc[index] + ".sorted.bam"
    !nohup macs2 callpeak -c $control -t $immunoprecipitate -n $name --outdir $outputdirectory & > $name.macs2.log

For an in-depth discussion of what MACS2 does: https://github.com/taoliu/MACS/wiki/Advanced:-Call-peaks-using-MACS2-subcommands

## Bedtools

In this tutorial, we'll use Bedtools to extract the intersecting regions of the MACS output between the experimental conditions.

The Bedtools suite of programs is widely used for genomic interval manipulation or "genome algebra". 

First we'll sort the output. The following line uses the `sort` command to sort the MACS output.

In [ ]:
for index, individual in enumerate(ChIPSeqTreatment):
    macs_output = "test/" + name + "/" + name + "_peaks.narrowPeak"
    sort = "test/" + name + "/" + name + "_peaks.narrowPeak.sorted"
    !sort -k 1,1 -k2,2n $macs_output > $sort

Then we'll find the intersecting regions between the different experimental conditions.

In [ ]:
!bedtools intersect -a test/SRR6703661/SRR6703661_peaks.narrowPeak.sorted -b test/SRR6703663/SRR6703663_peaks.narrowPeak.sorted -u > test/ChIPSeqintersect.bed

## Integrative Genomics Viewer

A BAM file viewer will allow you to see your reads in an interactive graphical display. There are many different viewers available such as UCSC Genome Browser, Integrative Genomics Viewer (IGV), and NCBI Genome Workbench.

To load .bam files please first do the following:

Transfer the data to your computer. Amazon provides documentation on copying files with scp (Linux, OSX) and WinSCP (Windows). 

For scp, the command will be approximately: scp ubuntu@your-instance-name-or-ip-address:/home/ubuntu/data/transcript.fa

Load annotation in IGV
File -> Load from File ... -> ChIPSeqintersect.bed

# RNA-Seq
## HTSeq (High-through sequencing)

HTSeq is a Python library to facilitate the rapid development of RNA-Seq analysis. HTSeq offers parsers for many common data formats in HTS projects, as well as classes to represent data, such as genomic coordinates, sequences, sequencing reads, alignments, gene model information and variant calls, and provides data structures that allow for querying via genomic coordinates. In this tutorial we will use htseq-count, a tool developed with HTSeq that preprocesses RNA-Seq data for differential expression analysis by counting the overlap of reads with genes.

In [ ]:
from pandas import read_csv

gtf = "data/Saccharomyces_cerevisiae.R64-1-1.84.gtf"

RNASeqSRARunTableFile='data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"]).astype(list)
RNASeqoutputSortBam = "test/" + RNASeqoutrun + ".sorted.bam"

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    input = individual
    output = individual + ".genecount.txt"
    !htseq-count -m intersection-nonempty -s no -f bam $input $gtf > $output

## DESeq (Differential Expression Sequencing)

Estimate variance-mean dependence in count data from high-throughput sequencing assays and test for differential expression based on a model using the negative binomial distribution

In [ ]:
!Rscript scripts/runDeseq.R

## Visualization

The following script performs principal component analysis and creates volcano plots and bar graphs of RNA-Seq expression.

In [ ]:
!Rscript scripts/loadYeastGeneCounts.R